# COMPSCI 589 HW1

#### Name: Haochen Wang

### SECTION 0: Load Libraries

In [1165]:
import sklearn.model_selection
import scipy
import numpy as np
import csv
import math
import matplotlib.pyplot as plt
from operator import itemgetter
from collections import Counter

### SECTION 2: Evaluating Decision Tree

In [1166]:
# Import Document

house_file = open('house_votes_84.csv', encoding='utf-8-sig')
csvreader = csv.reader(house_file)
houserows = []
for row in csvreader:
        houserows.append(row)

hvcat = houserows[0]
# print(housevotecat)

housevotedat = np.array([[int(element) for element in row] for row in houserows[1:]]).T

print(hvcat)

['handicapped-infants', 'water-project-cost-sharing', 'adoption-of-the-budget-resolution', 'physician-fee-freeze', 'el-salvador-adi', 'religious-groups-in-schools', 'anti-satellite-test-ban', 'aid-to-nicaraguan-contras', 'mx-missile', 'immigration', 'synfuels-corporation-cutback', 'education-spending', 'superfund-right-to-sue', 'crime', 'duty-free-exports', 'export-administration-act-south-africa', 'target']


In [1167]:
# Node Class
class Treenode:
    label = ""
    type = ""
    testattribute = ""
    
    

    def __init__(self, label, type):
        self.label = label
        self.type = type


In [1168]:
# Define helper functions that I use in decision tree.

def same(column):
    return all(item == column[0] for item in column)

def majority(column):
    return np.argmax(np.bincount(column))

def entropy(col):
    values = list(Counter(col).values())
    ent = 0
    for value in values:
        k = (value/sum(values))
        ent += -k*math.log(k,2)
    return ent

def gini(col):
    values = list(Counter(col).values())
    ginivalue = 1
    for value in values:
        prob = (value/sum(values))
        ginivalue -= prob**2
    return ginivalue

# test = np.array([1,1,3,3,11,11])
# same(test)
# majority(test)

# test1 = np.array([1,1,0,0,0]) # should get entropy .971, gini .48
# test2 = np.array([1,1,1,1,1,0,0,0,0,0,0,0,0,0]) # should get entropy .940, gini.459
# entropy(test2)

('physician-fee-freeze', 0.10197914174914247)

In [1170]:
# Define three test criteria: 
    
# ID3 - Entropy Gain 

def id3(collist, listattribution):
    original_ent = entropy(collist[-1])
    smallest_ent = 1
    i = 0
    # bestindex = i
    best = listattribution[i]
    for attributes in listattribution[:-1]: # I keep the last column: the target/label.
        liskey = list(Counter(collist[i]).keys())
        listofcategory = []
        for value in liskey:
            index = [idx for idx, element in enumerate(collist[i]) if element == value]
            category = np.array(collist[-1][index]) 
            listofcategory.append(category) # list of nparrays of target/label/categories.

        ent = 0
        for cat in listofcategory:
            a = len(cat)/len(collist[i]) # This is probability
            ent += a * entropy(cat) # probability multiple by entropy

        if ent < smallest_ent:
            smallest_ent = ent
            best = attributes
            # bestindex = i
        i+=1

    return best, original_ent-ent

# C4.5 - Entrophy Ratio 
# CART - Gini Impurity
def cart(collist, listattribution):
    smallest_gini = 1
    i = 0
    # bestindex = i
    best = listattribution[i]
    for attributes in listattribution[:-1]: # I keep the last column: the target/label.
        liskey = list(Counter(collist[i]).keys())
        listofcategory = []
        for value in liskey:
            index = [idx for idx, element in enumerate(collist[i]) if element == value]
            category = np.array(collist[-1][index]) 
            listofcategory.append(category) # list of nparrays of target/label/categories.

        gin = 0
        for cat in listofcategory:
            a = len(cat)/len(collist[i]) # This is probability
            gin += a * gini(cat) # probability multiple by gini

        if gin < smallest_gini:
            smallest_gini = gin
            best = attributes
            # bestindex = i
        i+=1

    return best, gin
    

In [1171]:
a_list = [1, 4, 49, 23, 4, 23]
output = [idx for idx, element in enumerate(a_list) if element == 4]
print(output)

[1, 4]


In [1172]:
# Decision Tree
def decisiontree(dataset, listattributes, type='id3'):
    def processbest():
        if type == "id3":
            return id3(dataset, listattributes)
        elif type == "cart":
            return cart(dataset, listattributes)
        else: 
            return cart(dataset, listattributes)

    node = Treenode("-1","decision")

    if same(dataset[-1]):
        node.type = "leaf"
        node.label = dataset[-1][0]
        return node
    
    if len(listattributes) == 0:
        node.type = "leaf"
        node.label = majority(dataset[-1])
        return node

    bestattribute = processbest()[0]
    node.testattribute = bestattribute
    bindex = dataset.index(bestattribute)

    bigv = Counter(dataset[bindex]).keys()
    

    listattributes.pop(bindex)
    dataset.pop(bindex)
    


    return

In [1177]:
t = [1,2,34,2,2,3,2,1,2,1,1,1,1,1,1,1,10]
Counter(t).keys()

dict_keys([1, 2, 34, 3, 10])